# Azure OpenAI Connection Test - System 2

This notebook tests:
1. Environment variable loading
2. Azure OpenAI client initialization
3. Basic LLM call
4. JSON response parsing
5. Materiality scoring format (pipeline simulation)

---

## Setup

Run this first to set up the environment and imports.

In [ ]:
import os
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv()

print("✅ Environment setup complete")
print(f"Project root: {project_root}")

In [ ]:
# Import the gai function and utilities
from shared.utils.utils import gai, initialize_client

print("✅ Imports successful")

---

## Test 1: Check Environment Variables

Verify that all required Azure OpenAI environment variables are set.

In [ ]:
print("="*80)
print("TEST 1: Environment Variables")
print("="*80)

# Check required variables
required_vars = {
    'AZURE_OPENAI_ENDPOINT': os.getenv('AZURE_OPENAI_ENDPOINT'),
    'AZURE_OPENAI_API_KEY': os.getenv('AZURE_OPENAI_API_KEY'),
}

# Check optional variables
optional_vars = {
    'AZURE_OPENAI_API_VERSION': os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview'),
    'AZURE_OPENAI_DEPLOYMENT': os.getenv('AZURE_OPENAI_DEPLOYMENT', 'gpt-4o-mini'),
}

# Display required vars
print("\nRequired variables:")
all_set = True
for var_name, var_value in required_vars.items():
    if var_value:
        # Mask the API key for security
        if 'KEY' in var_name:
            masked_value = var_value[:8] + '...' + var_value[-4:] if len(var_value) > 12 else '***'
            print(f"  ✅ {var_name}: {masked_value}")
        else:
            print(f"  ✅ {var_name}: {var_value}")
    else:
        print(f"  ❌ {var_name}: NOT SET")
        all_set = False

# Display optional vars
print("\nOptional variables (with defaults):")
for var_name, var_value in optional_vars.items():
    print(f"  {var_name}: {var_value}")

# Summary
if all_set:
    print("\n✅ TEST 1 PASSED: All required environment variables are set")
else:
    print("\n❌ TEST 1 FAILED: Missing required environment variables")
    print("\nPlease add to your .env file:")
    print("  AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/")
    print("  AZURE_OPENAI_API_KEY=your-azure-api-key-here")

---

## Test 2: Azure OpenAI Client Initialization

Test that we can successfully initialize the Azure OpenAI client.

In [ ]:
print("="*80)
print("TEST 2: Client Initialization")
print("="*80)

try:
    print("\nInitializing Azure OpenAI client (using environment variables)...")
    client = initialize_client(use_env_vars=True)
    
    print(f"✅ Client initialized successfully")
    print(f"Client type: {type(client).__name__}")
    print(f"\n✅ TEST 2 PASSED")
    
except Exception as e:
    print(f"❌ Client initialization failed")
    print(f"Error: {e}")
    print(f"\n❌ TEST 2 FAILED")
    import traceback
    traceback.print_exc()

---

## Test 3: Simple LLM Call

Test a basic call to Azure OpenAI to verify connectivity.

In [ ]:
print("="*80)
print("TEST 3: Simple LLM Call")
print("="*80)

try:
    print("\nSending test prompt to Azure OpenAI...")
    
    response = gai(
        sys_prompt="You are a helpful assistant.",
        user_prompt="Say 'Hello from Azure OpenAI!' and nothing else.",
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=True
    )
    
    print(f"\n✅ LLM call successful!")
    print(f"Response type: {type(response).__name__}")
    print(f"Response: {response}")
    print(f"\n✅ TEST 3 PASSED")
    
except Exception as e:
    print(f"\n❌ LLM call failed")
    print(f"Error: {e}")
    print(f"\n❌ TEST 3 FAILED")
    import traceback
    traceback.print_exc()

---

## Test 4: JSON Response Parsing

Test that JSON responses are correctly parsed.

In [ ]:
print("="*80)
print("TEST 4: JSON Response Parsing")
print("="*80)

try:
    print("\nRequesting JSON response from Azure OpenAI...")
    
    response = gai(
        sys_prompt="You are a helpful assistant that responds in JSON format.",
        user_prompt=(
            "Respond with a JSON object with the following fields:\n"
            "- status: 'success'\n"
            "- message: 'Azure OpenAI is working correctly'\n"
            "- test_passed: true\n"
            "Respond ONLY with the JSON object, no additional text."
        ),
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=True
    )
    
    print(f"\n✅ JSON parsing successful!")
    print(f"Response type: {type(response).__name__}")
    
    if isinstance(response, dict):
        print(f"\nParsed JSON:")
        for key, value in response.items():
            print(f"  - {key}: {value}")
        
        if response.get('status') == 'success':
            print(f"\n✅ TEST 4 PASSED: JSON validation successful")
        else:
            print(f"\n⚠️  TEST 4 PARTIAL: JSON received but validation failed")
    else:
        print(f"\n⚠️  Response was not parsed as JSON")
        print(f"Raw response: {response}")
        print(f"\n❌ TEST 4 FAILED")
    
except Exception as e:
    print(f"\n❌ JSON parsing test failed")
    print(f"Error: {e}")
    print(f"\n❌ TEST 4 FAILED")
    import traceback
    traceback.print_exc()

---

## Test 5: Materiality Scoring Format

Test the actual materiality scoring prompt format used in the pipeline.

In [ ]:
print("="*80)
print("TEST 5: Materiality Scoring Format")
print("="*80)

# Materiality scoring prompt (simplified version)
materiality_sys_prompt = """You are an expert analyst assessing the material impact of soft power events.

**YOUR TASK:**
Assign a materiality score from 1.0 to 10.0 measuring the concrete/substantive nature of this event.

**Scoring Scale:**
- 1-3: Symbolic/rhetorical (statements, cultural events with no material commitments)
- 4-6: Mixed symbolic and material (agreements with unclear implementation)
- 7-10: Highly material (concrete infrastructure, specific financial commitments)

**Output JSON format:**
{
    "material_score": 7.5,
    "justification": "Brief explanation"
}

Respond with ONLY the JSON object."""

materiality_user_prompt = """**Event:** China-Egypt Belt and Road Infrastructure Agreement
**Country:** China
**Time Period:** 2024-08-15
**Categories:** Economic, Infrastructure
**Recipients:** Egypt

**Event Description:**
China and Egypt signed a comprehensive infrastructure development agreement under the Belt and Road Initiative, 
including $2.5 billion in financing for a new high-speed rail link and port expansion project. 
Construction is scheduled to begin in Q1 2025."""

try:
    print("\nTesting materiality scoring prompt format...")
    
    response = gai(
        sys_prompt=materiality_sys_prompt,
        user_prompt=materiality_user_prompt,
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=True
    )
    
    print(f"\n✅ Materiality scoring test successful!")
    print(f"Response type: {type(response).__name__}")
    
    if isinstance(response, dict):
        score = response.get('material_score')
        justification = response.get('justification', '')
        
        print(f"\nParsed materiality response:")
        print(f"  - Material Score: {score}/10.0")
        print(f"  - Justification: {justification[:150]}..." if len(justification) > 150 else f"  - Justification: {justification}")
        
        if isinstance(score, (int, float)) and 1.0 <= score <= 10.0:
            print(f"\n✅ TEST 5 PASSED: Materiality scoring format validation successful")
            print(f"\n🎉 Your Azure OpenAI setup is ready for the materiality scoring pipeline!")
        else:
            print(f"\n⚠️  Score validation failed (expected 1.0-10.0, got {score})")
            print(f"\n❌ TEST 5 FAILED")
    else:
        print(f"\n⚠️  Response was not parsed as JSON")
        print(f"Raw response: {response}")
        print(f"\n❌ TEST 5 FAILED")
    
except Exception as e:
    print(f"\n❌ Materiality scoring test failed")
    print(f"Error: {e}")
    print(f"\n❌ TEST 5 FAILED")
    import traceback
    traceback.print_exc()

---

## Summary

Review all test results below.

In [ ]:
print("="*80)
print("TEST SUMMARY")
print("="*80)

print("""
If all tests passed:
  ✅ Your Azure OpenAI configuration is working correctly
  ✅ You can now run the summary and GAI pipeline on System 2
  ✅ The gai() function is ready to use with source="azure" and azure_use_env=True

Next steps:
  1. Update pipeline scripts to use: gai(..., source="azure", azure_use_env=True)
  2. Test specific pipeline components (bilateral summaries, event summaries, etc.)
  3. Run production workloads

If tests failed:
  - Review error messages above
  - Verify .env file configuration
  - Check Azure OpenAI resource settings
  - Ensure deployment name matches your Azure setup
""")

---

## Optional: Test AWS Secrets Manager Mode

If you want to test the AWS Secrets Manager credential approach instead, run this cell.

In [ ]:
print("="*80)
print("OPTIONAL: Testing AWS Secrets Manager Mode")
print("="*80)

try:
    print("\nAttempting to use AWS Secrets Manager for credentials...")
    
    response = gai(
        sys_prompt="You are a helpful assistant.",
        user_prompt="Say 'AWS Secrets Manager mode is working!' and nothing else.",
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=False  # Use AWS Secrets Manager
    )
    
    print(f"\n✅ AWS Secrets Manager mode successful!")
    print(f"Response: {response}")
    
except Exception as e:
    print(f"\n⚠️  AWS Secrets Manager mode failed (this is expected if not configured)")
    print(f"Error: {e}")
    print(f"\nThis is OK - you can use environment variables mode (azure_use_env=True) instead")